In [2]:
!pip install sentence_transformers

  Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.1 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 3.8 MB/s eta 0:00:00a 0:00:01
Using cached PyYAML-6.0.2-cp313-cp313-macosx_11_0_arm64.whl (171 kB)
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 4.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12.1
    Uninstalling sympy-1.12.1:
      Successfully uninstalled sympy-1.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pennylane-qiskit 0.41.0.post0 requires qiskit<1.3,>=0.32, but you have qiskit 1.4.2 which is incompatible.
pennylane-qiskit 0.41.0.post0 requires sympy<1.13, but you have sympy 1.14.0 which is incompatible.


In [2]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer

/Users/eric1026/qiskit/miniconda3/envs/qiskit/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# --- 1. 合併所有推文資料 ---
csv_files = sorted(glob.glob("data/tariff_data_en/*.csv"))
df_list = [pd.read_csv(f) for f in csv_files]
df = pd.concat(df_list, ignore_index=True)
df.head()

,Timestamp,Tweet Content,sentiment
0,2025-03-01T22:17:49.000Z,"Proposed tariffs on China, Canada, and Mexico ...",positive
1,2025-03-01T22:27:25.000Z,President Trump says his 25-percent tariff on ...,positive
2,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
3,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
4,2025-03-01T22:30:20.000Z,Beer Companies Say Trump’s Aluminum Tariff Wil...,negative


In [5]:
# --- 2. 標準化欄位與時間處理 ---
df.columns = ['Timestamp', 'text', 'sentiment']
df.dropna(subset=['text'], inplace=True)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # 含 UTC 時區
event_date = pd.to_datetime("2025-04-02").tz_localize("UTC")
df['period'] = df['Timestamp'].apply(lambda x: 'before' if x < event_date else 'after')

In [6]:
# --- 3. 使用預訓練 Sentence-BERT 將推文轉成語意向量 ---
model = SentenceTransformer('all-MiniLM-L6-v2')  # 輕量快速
X = model.encode(df['text'].tolist(), show_progress_bar=True)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Batches: 100%|██████████| 280/280 [00:21<00:00, 13.24it/s]


In [7]:
# --- 4. KMeans 聚類 ---
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# --- 5. PCA 降維 + 畫群體分布圖 ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

os.makedirs("saved_imgs", exist_ok=True)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], cmap='tab10', alpha=0.6)
plt.title("KMeans with SBERT Embedding (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(scatter, label='Cluster')
plt.savefig("saved_imgs/kmeans_pca_sbert.png")
plt.close()

In [11]:
# --- 6. 產出每個群的 WordCloud（照舊）---
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
custom_stopwords = set(['tariff', 'trump'])
all_stopwords = list(ENGLISH_STOP_WORDS.union(custom_stopwords))

def plot_cluster_wordcloud(df, cluster_num):
    text = " ".join(df[df['cluster'] == cluster_num]['text'])
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=all_stopwords).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Cluster {cluster_num} WordCloud")
    plt.savefig(f"saved_imgs/cluster_{cluster_num}_wordcloud_sbert.png")
    plt.close()

for cl in sorted(df['cluster'].unique()):
    plot_cluster_wordcloud(df, cl)

### 🧠 群體語意總覽分析

本研究以 Sentence-BERT 模型將推文轉換為語意向量，並透過 KMeans 聚類方法將語料分為六個主題群。經由每群高頻詞彙的詞雲（word cloud）視覺化與語意對照分析後，歸納出各群的核心語境與主題如下：

- **Cluster 0 – 美加貿易爭端與加拿大觀點**  
  該群體聚焦於加拿大與美國之間的貿易關係，頻繁出現的詞彙包含 `Canada`, `Trudeau`, `threat`, `tariffs`，顯示該群多為對美方政策持批評態度的推文，語氣偏向防禦與不滿。

- **Cluster 1 – 市場波動與投資反應**  
  此群包含大量與金融市場相關語彙，如 `market`, `stock`, `investor`, `recession`，反映出用戶對於貿易政策可能對經濟與股市造成影響的擔憂與分析，語氣呈現理性但偏焦慮。

- **Cluster 2 – 中美貿易戰與國際談判**  
  詞彙如 `China`, `export`, `deal`, `war` 明確指出該群主題為中美貿易衝突與談判進展，語氣多帶有對抗性或批判性，且事件後大量湧現，為本研究中最受事件影響的群體。

- **Cluster 3 – 關稅影響與消費經濟**  
  該群多出現 `cost`, `import`, `consumer`, `price`，代表對關稅政策對商品價格與消費者負擔的關注。語氣較為理性、經濟分析導向，屬於討論型內容。

- **Cluster 4 – 政策評論與官方言論反應**  
  含有 `President`, `policy`, `economy`, `administration` 等詞彙，代表該群集中在政府官員與政策制定者的發言評論，語氣可能因政治立場不同而分化，整體呈現評論性與新聞轉述性質。

- **Cluster 5 – 網民輿論與口語表態**  
  本群聚焦於網路社群中的一般輿論，詞彙如 `just`, `say`, `like`, `want` 表現出明顯的非正式、情緒化用語，語氣較為自由、多樣，並包含支持、諷刺與戲謔等多元觀點，是情緒最偏向正面的群體。

整體而言，分群結果有效劃分了不同語意主題，並揭示了政策事件如何在語意與情緒層面觸發特定群體的反應，為理解社群輿


### 📋 群體語意總覽表格

| Cluster | 主題焦點                         | 高頻關鍵詞（部分）                           | 語氣傾向         | 備註                             |
|---------|----------------------------------|----------------------------------------------|------------------|----------------------------------|
| 0       | 美加貿易爭端與加拿大觀點         | Canada, Trudeau, threat, tariffs, US         | 負面、批判       | 舊議題為主，事件前集中出現       |
| 1       | 市場波動與投資反應               | market, stock, investor, recession, risk     | 焦慮、理性       | 與金融市場連動，反應全球不安     |
| 2       | 中美貿易戰與國際談判             | China, deal, war, export, negotiation        | 對立、憤怒       | 事件後激增，情緒最負面           |
| 3       | 關稅影響與消費經濟               | cost, import, consumer, price, impact        | 中性、理性       | 關注稅收與價格影響               |
| 4       | 政策評論與官方言論反應           | President, policy, administration, economy   | 評論、兩極       | 涉及領導人與政府發言             |
| 5       | 網民輿論與口語表態               | just, like, say, want, week, new             | 多元、偏正向     | 社群用語多，情緒最活躍           |


In [10]:
# --- 7. 額外分析輸出：事件前後各群數量與情緒分布 ---
print("\n📊 各群情緒比例（row-normalized）:")
print(df.groupby('cluster')['sentiment'].value_counts(normalize=True).unstack().fillna(0))

print("\n📆 各群在事件前/後的比例（row-normalized）:")
print(pd.crosstab(df['cluster'], df['period'], normalize='index'))


📊 各群情緒比例（row-normalized）:
sentiment  negative   neutral  positive
cluster                                
0          0.501894  0.174242  0.323864
1          0.536533  0.127507  0.335960
2          0.549412  0.164706  0.285882
3          0.380933  0.251810  0.367257
4          0.504292  0.194421  0.301288
5          0.425767  0.164417  0.409816

📆 各群在事件前/後的比例（row-normalized）:
period      after    before
cluster                    
0        0.089015  0.910985
1        0.393266  0.606734
2        0.755294  0.244706
3        0.446903  0.553097
4        0.443348  0.556652
5        0.461350  0.538650


## 分群情緒傾向與事件前後分布分析 （GPT）

為探討推文內容在語意與情緒上的異質性，我們使用 Sentence-BERT 嵌入向量與 KMeans 聚類對語料進行分群（共六群），並進一步分析各群的情緒比例及其在關稅政策事件（2025 年 4 月 2 日）前後的出現頻率。下列為各群重點分析結果：

1. 情緒傾向分析
透過原始資料中的情緒標註（positive、neutral、negative），我們計算了各群體內部的情緒分布。結果顯示，群 0、1、2 的負面情緒比例皆超過 50%，反映其內容多涉及爭議性話題或批判性語言。特別是群 2，負面情緒高達 54.9%，其主題以中美貿易戰為主，詞彙如 "Chinese", "war", "export", "negotiation" 明顯帶有強烈對立語意。

相較之下，群 3 與群 5 呈現相對中性或偏向正向情緒，其中群 5 的正向情緒比例最高（約 40.9%），顯示該群文本多以網民口語評論為主，語氣較為自由且表達支持、讚許等情緒的比例較高。群 3 則多與消費者成本、進口商品相關，語氣相對理性與分散。

2. 事件前後出現比例分析
對於事件時間（2025-04-02）前後的分布，我們發現各群反應明顯不同。群 2 的推文有超過 75% 發生於事件之後，顯示該主題與該事件（如新一輪關稅政策、重大外交爭端）有高度相關性，並引發大量後續討論。相對而言，群 0 的推文 91% 發生在事件之前，推測與早期美加貿易爭端相關的議題熱度已逐漸退燒。

其他群（群 1、3、4、5）則呈現出較為平均的分布，顯示這些主題在事件前後均有持續性討論。例如群 1（市場波動與投資）、群 4（政府政策與總統發言）皆為長期關注議題，並非僅由特定事件引發。

3. 總結
綜合語意與時間分析可知，本次事件最直接影響的群體為 Cluster 2，其不僅情緒傾向偏負面，亦在事件後大量增加。此結果突顯語意分群能有效區分出對政策事件敏感的主題群，為輿情變化監控與情緒預測提供依據。未來可考慮進一步追蹤此類事件引發的次波輿論走向，並結合情緒強度與主題動態模型以強化分析深度。



## 0404 (美股重挫)

In [11]:
# --- 1. 合併所有推文資料 ---
csv_files = sorted(glob.glob("data/tariff_data_en/*.csv"))
df_list = [pd.read_csv(f) for f in csv_files]
df = pd.concat(df_list, ignore_index=True)
df.head()

,Timestamp,Tweet Content,sentiment
0,2025-03-01T22:17:49.000Z,"Proposed tariffs on China, Canada, and Mexico ...",positive
1,2025-03-01T22:27:25.000Z,President Trump says his 25-percent tariff on ...,positive
2,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
3,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
4,2025-03-01T22:30:20.000Z,Beer Companies Say Trump’s Aluminum Tariff Wil...,negative


In [13]:
# --- 2. 標準化欄位與時間處理 ---
df.columns = ['Timestamp', 'text', 'sentiment']
df.dropna(subset=['text'], inplace=True)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # 含 UTC 時區
event_date = pd.to_datetime("2025-04-04").tz_localize("UTC")
df['period'] = df['Timestamp'].apply(lambda x: 'before' if x < event_date else 'after')

In [14]:
# --- 3. 使用預訓練 Sentence-BERT 將推文轉成語意向量 ---
model = SentenceTransformer('all-MiniLM-L6-v2')  # 輕量快速
X = model.encode(df['text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 280/280 [00:13<00:00, 21.17it/s]


In [15]:
# --- 4. KMeans 聚類 ---
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

In [16]:
# --- 5. PCA 降維 + 畫群體分布圖 ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

os.makedirs("saved_imgs_0404", exist_ok=True)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], cmap='tab10', alpha=0.6)
plt.title("KMeans with SBERT Embedding (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(scatter, label='Cluster')
plt.savefig("saved_imgs_0404/kmeans_pca_sbert.png")
plt.close()

In [18]:
# --- 6. 產出每個群的 WordCloud（照舊）---
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
custom_stopwords = set(['tariff', 'trump','stock'])
all_stopwords = list(ENGLISH_STOP_WORDS.union(custom_stopwords))

def plot_cluster_wordcloud(df, cluster_num):
    text = " ".join(df[df['cluster'] == cluster_num]['text'])
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=all_stopwords).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Cluster {cluster_num} WordCloud")
    plt.savefig(f"saved_imgs_0404/cluster_{cluster_num}_wordcloud_sbert.png")
    plt.close()

for cl in sorted(df['cluster'].unique()):
    plot_cluster_wordcloud(df, cl)

In [19]:
# --- 7. 額外分析輸出：事件前後各群數量與情緒分布 ---
print("\n📊 各群情緒比例（row-normalized）:")
print(df.groupby('cluster')['sentiment'].value_counts(normalize=True).unstack().fillna(0))

print("\n📆 各群在事件前/後的比例（row-normalized）:")
print(pd.crosstab(df['cluster'], df['period'], normalize='index'))


📊 各群情緒比例（row-normalized）:
sentiment  negative   neutral  positive
cluster                                
0          0.501894  0.174242  0.323864
1          0.536533  0.127507  0.335960
2          0.549412  0.164706  0.285882
3          0.380933  0.251810  0.367257
4          0.504292  0.194421  0.301288
5          0.425767  0.164417  0.409816

📆 各群在事件前/後的比例（row-normalized）:
period      after    before
cluster                    
0        0.087121  0.912879
1        0.376074  0.623926
2        0.749412  0.250588
3        0.417136  0.582864
4        0.419742  0.580258
5        0.446626  0.553374


### 🌟 亮點觀察總結（事件為 4/4 股市暴跌）

- **Cluster 2：事件後激增，反映中美貿易與股市連動恐慌**  
  該群以 `China`, `war`, `tariffs`, `deal`, `talk` 為核心詞彙，聚焦中美衝突與政策交鋒。  
  有 **74.9% 推文出現在事件後**，且 **負面情緒高達 54.9%**，是對 4/4 市場衝擊反應最直接的一群。

- **Cluster 0：以加拿大視角討論貿易政策，歷史成分較高**  
  關鍵詞如 `Canada`, `Trudeau`, `threat`, `tariffs`, `American` 指出其焦點在加美貿易爭端，  
  **91.3% 發表於事件前**，顯示此為早期議題，與 4/4 股災相關性不高。

- **Cluster 1：投資人觀點顯著，展現市場憂慮與波動預期**  
  以 `market`, `stock`, `investor`, `recession`, `Bitcoin` 等詞為主，語氣偏焦慮。  
  **37.6% 推文來自事件後**，且 **負面情緒為 53.7%**，反映金融市場對利空消息高度敏感。

- **Cluster 3：價格與消費層面討論，具經濟理性分析特徵**  
  詞彙如 `price`, `cost`, `consumer`, `import`, `pay`，語氣較中性、分析導向。  
  雖 **41.7% 為事件後貼文**，但其負面情緒比例僅為 38.1%，屬於情緒較平衡的群體。

- **Cluster 4：集中政府言論與宏觀政策評論，語氣偏政治性**  
  高頻詞如 `President`, `plan`, `policy`, `economy`, `Donald`，反映政治領導與經濟戰略討論。  
  約 **42% 推文為事件後**，負面情緒也達 **50.4%**，代表政經議題在股災後仍具討論熱度。

- **Cluster 5：社群語言活躍，口語化與情緒波動明顯**  
  使用 `just`, `war`, `day`, `like`, `President`, `threat` 等詞彙，帶有即時評論與網民語感。  
  為正面情緒最高群（**40.9%**），**44.7% 為事件後貼文**，可能包含幽默、戲謔或支持發言。


#### Stock market related

In [20]:
import pandas as pd
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# Step 1：篩選事件當天的推文
event_date = pd.to_datetime("2025-04-04").tz_localize("UTC")
same_day_df = df[df['Timestamp'].dt.date == event_date.date()]

# Step 2：使用 CountVectorizer 抽取單字頻率
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X_count = vectorizer.fit_transform(same_day_df['text'])

# Step 3：統計最常出現的詞
word_counts = X_count.sum(axis=0).A1
vocab = vectorizer.get_feature_names_out()
top_words = pd.DataFrame({'word': vocab, 'count': word_counts})
top_words = top_words.sort_values(by='count', ascending=False).head(30)

# Step 4：可視化（選擇性）
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.barh(top_words['word'][::-1], top_words['count'][::-1])
plt.title("🔍 Top Keywords on 2025-04-04 (Stock Plunge Day)")
plt.xlabel("Frequency")
plt.tight_layout()
plt.savefig("saved_imgs_0404/event_day_top_words.png")
plt.close()


/var/folders/xw/sscnc6jx4kb4klgrdr8sgyz80000gn/T/ipykernel_4748/1006423659.py:25: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
/var/folders/xw/sscnc6jx4kb4klgrdr8sgyz80000gn/T/ipykernel_4748/1006423659.py:26: UserWarning: Glyph 128269 (\N{LEFT-POINTING MAGNIFYING GLASS}) missing from font(s) DejaVu Sans.
  plt.savefig("saved_imgs_0404/event_day_top_words.png")


## 0409 (額外關稅第一波施加)


In [3]:
# --- 1. 合併所有推文資料 ---
csv_files = sorted(glob.glob("data/tariff_data_en/*.csv"))
df_list = [pd.read_csv(f) for f in csv_files]
df = pd.concat(df_list, ignore_index=True)
df.head()

,Timestamp,Tweet Content,sentiment
0,2025-03-01T22:17:49.000Z,"Proposed tariffs on China, Canada, and Mexico ...",positive
1,2025-03-01T22:27:25.000Z,President Trump says his 25-percent tariff on ...,positive
2,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
3,2025-03-01T23:00:01.000Z,"""You should pay less for someone that has tari...",negative
4,2025-03-01T22:30:20.000Z,Beer Companies Say Trump’s Aluminum Tariff Wil...,negative


In [4]:
# --- 2. 標準化欄位與時間處理 ---
df.columns = ['Timestamp', 'text', 'sentiment']
df.dropna(subset=['text'], inplace=True)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])  # 含 UTC 時區
event_date = pd.to_datetime("2025-04-09").tz_localize("UTC")
df['period'] = df['Timestamp'].apply(lambda x: 'before' if x < event_date else 'after')

In [5]:
# --- 3. 使用預訓練 Sentence-BERT 將推文轉成語意向量 ---
model = SentenceTransformer('all-MiniLM-L6-v2')  # 輕量快速
X = model.encode(df['text'].tolist(), show_progress_bar=True)

Batches: 100%|██████████| 280/280 [00:17<00:00, 15.97it/s]


In [6]:
# --- 4. KMeans 聚類 ---
k = 6
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:
# --- 5. PCA 降維 + 畫群體分布圖 ---
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

os.makedirs("saved_imgs_0409", exist_ok=True)

plt.figure(figsize=(10, 7))
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=df['cluster'], cmap='tab10', alpha=0.6)
plt.title("KMeans with SBERT Embedding (PCA)")
plt.xlabel("PCA 1")
plt.ylabel("PCA 2")
plt.colorbar(scatter, label='Cluster')
plt.savefig("saved_imgs_0409/kmeans_pca_sbert.png")
plt.close()

In [9]:
# --- 6. 產出每個群的 WordCloud（照舊）---
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
custom_stopwords = set(['tariff', 'trump'])
all_stopwords = list(ENGLISH_STOP_WORDS.union(custom_stopwords))

def plot_cluster_wordcloud(df, cluster_num):
    text = " ".join(df[df['cluster'] == cluster_num]['text'])
    wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=all_stopwords).generate(text)
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title(f"Cluster {cluster_num} WordCloud")
    plt.savefig(f"saved_imgs_0409/cluster_{cluster_num}_wordcloud_sbert.png")
    plt.close()

for cl in sorted(df['cluster'].unique()):
    plot_cluster_wordcloud(df, cl)

In [10]:
# --- 7. 額外分析輸出：事件前後各群數量與情緒分布 ---
print("\n📊 各群情緒比例（row-normalized）:")
print(df.groupby('cluster')['sentiment'].value_counts(normalize=True).unstack().fillna(0))

print("\n📆 各群在事件前/後的比例（row-normalized）:")
print(pd.crosstab(df['cluster'], df['period'], normalize='index'))


📊 各群情緒比例（row-normalized）:
sentiment  negative   neutral  positive
cluster                                
0          0.501894  0.174242  0.323864
1          0.536533  0.127507  0.335960
2          0.549412  0.164706  0.285882
3          0.380933  0.251810  0.367257
4          0.504292  0.194421  0.301288
5          0.425767  0.164417  0.409816

📆 各群在事件前/後的比例（row-normalized）:
period      after    before
cluster                    
0        0.072917  0.927083
1        0.315903  0.684097
2        0.652941  0.347059
3        0.327836  0.672164
4        0.333047  0.666953
5        0.391411  0.608589


### 📋 群體語意總覽表格（整合語意、情緒與時間）

| Cluster | 主題焦點                         | 高頻關鍵詞（部分）                           | 語氣傾向       | 負面情緒比例 | 事件後佔比 | 備註                             |
|---------|----------------------------------|----------------------------------------------|----------------|----------------|------------|----------------------------------|
| 0       | 美加貿易爭端與加拿大觀點         | Canada, Trudeau, threat, tariffs, product    | 批判、防禦型   | 50.2%         | 7.3%       | 幾乎皆在事件前出現，為歷史議題   |
| 1       | 市場波動與投資反應               | stock, market, investor, inflation, Bitcoin  | 焦慮、理性型   | 53.7%         | 31.6%      | 關注股市與金融衝擊               |
| 2       | 中美貿易戰與國際談判             | China, deal, war, export, talk               | 對立、批判型   | 54.9%         | 65.3%      | 與事件高度相關，事件後激增       |
| 3       | 關稅影響與消費經濟               | tariffs, price, cost, consumer, product      | 中性、理性型   | 38.1%         | 32.8%      | 涉及民生物價與進出口稅收         |
| 4       | 政策評論與官方言論反應           | President, economy, policy, plan             | 評論性混合型   | 50.4%         | 33.3%      | 涉及政府發言與政策方向           |
| 5       | 網民輿論與社群口語反應           | just, like, say, new, talk, war              | 多元、情緒化   | 42.6%         | 39.1%      | 社群意見自由發表，情緒波動大     |


### 🌟 亮點觀察總結

- **Cluster 2 是事件後反應最強烈的群體**  
  此群以「中美貿易戰」為核心主題，包含大量如 `China`, `war`, `deal`, `export` 等對立性詞彙。  
  推文中有 **超過 65% 出現在事件後**，且 **負面情緒佔比高達 54.9%**，顯示事件可能引發了大量不安與批判言論。

- **Cluster 0 幾乎全部為事件前的歷史議題**  
  聚焦「美加貿易爭端」與加拿大角度（詞彙如 `Canada`, `Trudeau`, `threat`），  
  **事件後僅佔 7.3%**，說明此主題可能已逐漸降溫，且討論集中於過往政策與影響。

- **Cluster 5 為情緒最正向的群體，語言風格偏社群與口語化**  
  常見詞彙如 `just`, `like`, `say`, `want`，顯示用語自然、輕鬆，符合社交平台特徵。  
  **正向情緒佔比達 40.9%**，為六群之最，呈現輿論自由表達與部分幽默或支持性語氣。

- **Cluster 3 討論焦點偏向經濟負擔與消費者角度**  
  關注如 `cost`, `price`, `consumer`, `impact` 等詞彙，語氣整體偏中性與理性。  
  雖不屬於情緒極端群體，但也具備長期討論價值，反映政策對生活層面的實際影響。

- **Cluster 1 表現出對市場的焦慮與觀望情緒**  
  主題集中於 `stock`, `market`, `investor`, `recession` 等詞，  
  並有 **53.7% 的負面情緒**，顯示經濟不確定性引發的投資憂慮普遍存在。

- **Cluster 4 涉及政府與領導人言論，情緒兩極化**  
  高頻詞包含 `President`, `policy`, `plan`, `economy`，涉及官方發言與政見。  
  雖分布穩定，但情緒容易隨立場變化而分化，呈現評論性與對立性混合語氣。
